In [1]:
import brightway2 as bw
import numpy as np
import json, os

#Local files
from import_databases import *

In [2]:
with open('global_settings.json', 'rb') as f:
    settings = json.load(f)
which_pc = settings['which_pc']

### TODO -> give paths to databases files

In [3]:
if which_pc == 'local':
    # Older databases
    ei33_path = '/Users/akim/Documents/LCA_files/ecoinvent 3.3 cutoff/datasets'  
    ex22_path = '/Users/akim/Documents/LCA_files/exiobase 2.2/mrIOT_IxI_fpa_coefficient_version2.2.2'
    ag12_path = '/Users/akim/Documents/LCA_files/agribalyse 1.2/AGRIBALYSEv1.2_Complete.xml'
    # Newer databases
    ei36_path = '/Users/akim/Documents/LCA_files/ecoinvent 3.6 cutoff/datasets'  
    # ex34_path = '/Users/akim/Documents/LCA_files/exiobase 3.4/IOT_2011_pxp/' #TODO
    ag13_path = '/Users/akim/Documents/LCA_files/agribalyse 1.3/Agribalyse CSV FINAL_no links_Nov2016v3.CSV'
    habe_path = '/Users/akim/Documents/LCA_files/HABE_2017/'
    co_path = 'data/es8b01452_si_002.xlsx'
elif which_pc == 'merlin':
    ex22_path = '/data/user/kim_a/LCA_files/exiobase_22/mrIOT_IxI_fpa_coefficient_version2.2.2'
    ei33_path = '/data/user/kim_a/LCA_files/ecoinvent_33_cutoff/datasets'  
    ei36_path = '/data/user/kim_a/LCA_files/ecoinvent_36_cutoff/datasets'  
    ag13_path = '/data/user/kim_a/LCA_files/agribalyse_13/Agribalyse CSV FINAL_no links_Nov2016v3.CSV'
    habe_path = '/data/user/kim_a/LCA_files/HABE_2017'
    co_path = 'data/es8b01452_si_002.xlsx'

### TODO -> choose project

In [6]:
project = 'GSA for paper'
bw.projects.set_current(project)
# del bw.databases['CH consumption 1.0']

### Import necessary databases

In [7]:
%%time 
ei36_name = 'ecoinvent 3.6 cutoff'
co_name = CONSUMPTION_DB_NAME

if project == 'GSA for ecoinvent':
    bw.bw2setup()
    import_exiobase_22(ex22_path)
    import_ecoinvent(ei36_path, ei36_name)
    import_agribalyse_13(ag13_path, ei36_name)
    # Consumption DB
    create_ecoinvent_33_project(ei33_path)
    import_consumption_db(co_path, habe_path, exclude_dbs=['heia'])
    add_consumption_activities(co_name, habe_path)

elif project == 'GSA for paper':
    bw.bw2setup()
    import_ecoinvent(ei36_path, ei36_name)
    # Consumption DB
    create_ecoinvent_33_project(ei33_path)
    exclude_dbs = [
        'heia', 
        'EXIOBASE 2.2',
        'Agribalyse 1.2',
        'Agribalyse 1.3 - ecoinvent 3.6 cutoff',
    ]
    import_consumption_db(co_path, habe_path, exclude_dbs=exclude_dbs)
    add_consumption_activities(co_name, habe_path)
    add_consumption_categories(co_name, co_path)
    add_consumptionn_sectors(co_name)
    
elif project == 'GSA no exiobase':
    bw.bw2setup()
    import_ecoinvent(ei36_path, ei36_name)
    import_agribalyse_13(ag13_path, ei36_name)
    # Consumption DB
    create_ecoinvent_33_project(ei33_path)
    exclude_dbs = [
        'heia', 
        'EXIOBASE 2.2',
    ]
    import_consumption_db(co_path, habe_path, exclude_dbs=exclude_dbs)
    add_consumption_activities(co_name, habe_path)
    
elif project == 'GSA for setac':
    bw.bw2setup()
    import_ecoinvent(ei36_path, ei36_name)
    # Consumption DB
    create_ecoinvent_33_project(ei33_path)
    exclude_dbs = [
        'heia', 
        'EXIOBASE 2.2',
        'Agribalyse 1.2',
        'Agribalyse 1.3 - ecoinvent 3.6 cutoff',
    ]
    import_consumption_db(co_path, habe_path, exclude_dbs=exclude_dbs)
    add_consumption_activities(co_name, habe_path)
    add_consumption_categories(co_name, co_path)
    add_consumptionn_sectors(co_name)

Biosphere database already present!!! No setup is needed
ecoinvent 3.6 cutoff database already present!!! No import is needed
Biosphere database already present!!! No setup is needed
ecoinvent 3.3 cutoff database already present!!! No import is needed
CH consumption 1.0 database already present!!! No import is needed
9734
'ch hh average consumption' (1 month of consumption, CH, None)
CPU times: user 16.6 s, sys: 4.94 s, total: 21.6 s
Wall time: 30.1 s


In [8]:
co = bw.Database('CH consumption 1.0')
act = co.search('average')[0]
demand = {act: 1}
method = ('IPCC 2013', 'climate change', 'GTP 100a')
act

'ch hh average consumption' (1 month of consumption, CH, None)

In [9]:
%%time
lca = bw.LCA(demand, method)
lca.lci()
lca.lcia()
lca.score

CPU times: user 1.82 s, sys: 129 ms, total: 1.95 s
Wall time: 769 ms


1039.2100378550203

In [10]:
act = co.search('average')[0]
list(act.exchanges())

[Exchange: 1 1 month of consumption 'ch hh average consumption' (1 month of consumption, CH, None) to 'ch hh average consumption' (1 month of consumption, CH, None)>,
 Exchange: 201.46845215493326 CHF 'Other insurance premiums' (CHF, CH, None) to 'ch hh average consumption' (1 month of consumption, CH, None)>,
 Exchange: 76.54080218942863 CHF 'Fees' (CHF, CH, None) to 'ch hh average consumption' (1 month of consumption, CH, None)>,
 Exchange: 46.03483906003794 kilogram 'Bakery products (sweet and salty)' (kilogram, CH, None) to 'ch hh average consumption' (1 month of consumption, CH, None)>,
 Exchange: 2.9620356479863883 kilogram 'Sandwich' (kilogram, CH, None) to 'ch hh average consumption' (1 month of consumption, CH, None)>,
 Exchange: 9.129392882033821 kilogram 'Fresh eggs' (kilogram, CH, None) to 'ch hh average consumption' (1 month of consumption, CH, None)>,
 Exchange: 0.8928952539848366 kilogram 'Processed eggs' (kilogram, CH, None) to 'ch hh average consumption' (1 month of co